<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/12-Improve_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables

In [1]:
!pip install -q llama-index==0.10.11 openai==1.12.0 llama-index-finetuning llama-index-embeddings-huggingface llama-index-readers-web tiktoken==0.6.0 chromadb==0.4.22 pandas==2.2.0 html2text sentence_transformers pydantic kaleido==0.2.1

In [1]:
import os

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"

In [2]:
# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.

import nest_asyncio

nest_asyncio.apply()

# Load a Model

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(temperature=0.9, model="gpt-3.5-turbo-0125", max_tokens=512)

/Users/louis/Documents/GitHub/ai-tutor-rag-system/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create a VectoreStore

In [5]:
import chromadb

# create client and a new collection
# chromadb.EphemeralClient saves data in-memory.
chroma_client = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = chroma_client.create_collection("mini-llama-articles")

In [7]:
from llama_index.vector_stores.chroma import ChromaVectorStore

# Define a storage context object using the created vector database.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Load the Dataset (CSV)

## Download

The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model. Read the dataset as a long string.

In [8]:
!curl -o ./mini-llama-articles.csv https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  169k  100  169k    0     0   915k      0 --:--:-- --:--:-- --:--:--  911k


## Read File

In [9]:
import csv

rows = []

# Load the file as a JSON
with open("./mini-llama-articles.csv", mode="r", encoding="utf-8") as file:
  csv_reader = csv.reader(file)

  for idx, row in enumerate( csv_reader ):
    if idx == 0: continue; # Skip header row
    rows.append( row )

# The number of characters in the dataset.
len( rows )

14

# Convert to Document obj

In [10]:
from llama_index.core import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [Document(text=row[1], metadata={"title": row[0], "url": row[2], "source_name": row[3]}) for row in rows]

# Transforming

In [11]:
from llama_index.core.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=512, chunk_overlap=128
)

In [12]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        QuestionsAnsweredExtractor(questions=3, llm=llm),
        SummaryExtractor(summaries=["prev", "self"], llm=llm),
        KeywordExtractor(keywords=10, llm=llm),
        OpenAIEmbedding(),
    ],
    vector_store=vector_store
)

nodes = pipeline.run(documents=documents, show_progress=True);

Parsing nodes:   0%|          | 0/14 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 108/108 [00:02<00:00, 38.77it/s]


In [13]:
len( nodes )

108

In [14]:
!zip -r vectorstore.zip mini-llama-articles

updating: mini-llama-articles/ (stored 0%)
updating: mini-llama-articles/chroma.sqlite3 (deflated 65%)
  adding: mini-llama-articles/aaac4d54-4f82-40da-b769-a6aecfa59eb0/ (stored 0%)
  adding: mini-llama-articles/aaac4d54-4f82-40da-b769-a6aecfa59eb0/data_level0.bin (deflated 96%)
  adding: mini-llama-articles/aaac4d54-4f82-40da-b769-a6aecfa59eb0/length.bin (deflated 35%)
  adding: mini-llama-articles/aaac4d54-4f82-40da-b769-a6aecfa59eb0/link_lists.bin (stored 0%)
  adding: mini-llama-articles/aaac4d54-4f82-40da-b769-a6aecfa59eb0/header.bin (deflated 61%)


# Load Indexes

In [16]:
# !unzip vectorstore.zip

In [17]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

# Create your index
db = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = db.get_or_create_collection("mini-llama-articles")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [18]:
# Create your index
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_vector_store(vector_store)

# Multi-Step Query Engine

## GPT-4

In [19]:
from llama_index.core import ServiceContext

gpt4 = OpenAI(temperature=0, model="gpt-4-0125-preview")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

/var/folders/l7/9qcp7g5x5rl9x8ltw0t85qym0000gn/T/ipykernel_1424/226941912.py:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)


In [20]:
from llama_index.core.indices.query.query_transform.base import StepDecomposeQueryTransform

step_decompose_transform_gpt4 = StepDecomposeQueryTransform(llm=gpt4, verbose=True)

In [21]:
from llama_index.core.query_engine.multistep_query_engine import MultiStepQueryEngine

query_engine_gpt4 = vector_index.as_query_engine(service_context=service_context_gpt4)
query_engine_gpt4 = MultiStepQueryEngine(
    query_engine=query_engine_gpt4,
    query_transform=step_decompose_transform_gpt4,
    index_summary="Used to answer questions about the LLaMA2 Model",
)

# Query Dataset

## Default

In [22]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
query_engine = vector_index.as_query_engine()

res = query_engine.query("How many parameters LLaMA2 model has?")

In [23]:
res.response

'The Llama 2 model is available in four different sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.'

In [24]:
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("Text\t", src.text)
  print("Score\t", src.score)
  print("-_"*20)

Node ID	 63380d3f-7aff-47cd-b2c1-e4baaed70a7e
Title	 Meta's Llama 2: Revolutionizing Open Source Language Models for Commercial Use
Text	 I. Llama 2: Revolutionizing Commercial Use Unlike its predecessor Llama 1, which was limited to research use, Llama 2 represents a major advancement as an open-source commercial model. Businesses can now integrate Llama 2 into products to create AI-powered applications. Availability on Azure and AWS facilitates fine-tuning and adoption. However, restrictions apply to prevent exploitation. Companies with over 700 million active daily users cannot use Llama 2. Additionally, its output cannot be used to improve other language models.  II. Llama 2 Model Flavors Llama 2 is available in four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters. While 7B, 13B, and 70B have already been released, the 34B model is still awaited. The pretrained variant, trained on a whopping 2 trillion tokens, boasts a context window of 4096 toke

## GPT-4 Multi-Step

In [25]:
response_gpt4 = query_engine_gpt4.query("How many parameters LLaMA2 model has?")

> Current query: How many parameters LLaMA2 model has?
> New query: What is the LLaMA2 Model?
> Current query: How many parameters LLaMA2 model has?
> New query: None


In [26]:
response_gpt4.response

'The LLaMA2 model has parameters ranging from 7 billion to 70 billion.'

In [27]:
for src in response_gpt4.source_nodes:
  print("Node ID\t", src.node_id)
  print("Text\t", src.text)
  print("Score\t", src.score)
  print("-_"*20)

Node ID	 3f7709ed-985e-417f-b88d-e3eac6ae8a06
Text	 
Question: What is the LLaMA2 Model?
Answer: The Llama 2 model is an open-source commercial language model developed by Meta, available in different sizes ranging from 7 billion to 70 billion parameters. It is designed to be integrated into AI-powered applications for businesses, with a focus on safety considerations in its design. The model's Ghost Attention feature enhances conversational continuity, and it possesses a groundbreaking temporal capability for organizing information based on time relevance to deliver contextually accurate responses.
Score	 None
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Node ID	 63380d3f-7aff-47cd-b2c1-e4baaed70a7e
Text	 I. Llama 2: Revolutionizing Commercial Use Unlike its predecessor Llama 1, which was limited to research use, Llama 2 represents a major advancement as an open-source commercial model. Businesses can now integrate Llama 2 into products to create AI-powered applications. Availability on A

# Test GPT-3 Multi-Step

In [28]:
from llama_index.core import ServiceContext
from llama_index.core.indices.query.query_transform.base import StepDecomposeQueryTransform
from llama_index.core.query_engine.multistep_query_engine import MultiStepQueryEngine

gpt3 = OpenAI(temperature=0, model="gpt-3.5-turbo-0125")
service_context_gpt3 = ServiceContext.from_defaults(llm=gpt3)

step_decompose_transform_gpt3 = StepDecomposeQueryTransform(llm=gpt3, verbose=True)

query_engine_gpt3 = vector_index.as_query_engine(service_context=service_context_gpt3)
query_engine_gpt3 = MultiStepQueryEngine(
    query_engine=query_engine_gpt3,
    query_transform=step_decompose_transform_gpt3,
    index_summary="Used to answer questions about the LLaMA2 Model",
)

/var/folders/l7/9qcp7g5x5rl9x8ltw0t85qym0000gn/T/ipykernel_1424/1136257440.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context_gpt3 = ServiceContext.from_defaults(llm=gpt3)


In [29]:
response_gpt3 = query_engine_gpt3.query("How many parameters LLaMA2 model has?")

> Current query: How many parameters LLaMA2 model has?
> New query: What are the main components or features of the LLaMA2 model?
> Current query: How many parameters LLaMA2 model has?
> New query: What is the range of model sizes available for the LLaMA2 model?
> Current query: How many parameters LLaMA2 model has?
> New query: What are the safety considerations in the LLaMA2 model?


In [30]:
response_gpt3.response

'The LLaMA2 model has model sizes ranging from 7 billion to 70 billion parameters.'

# Test Retriever on Multistep

In [31]:
import llama_index

In [32]:
from llama_index.core.indices.query.schema import QueryBundle

In [33]:
t = QueryBundle("How many parameters LLaMA2 model has?")

In [34]:
query_engine_gpt3.retrieve(t)

NotImplementedError: This query engine does not support retrieve, use query directly

# HyDE Transform

In [35]:
query_engine = vector_index.as_query_engine()

In [36]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine.transform_query_engine import TransformQueryEngine

hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)

In [37]:
response = hyde_query_engine.query("How many parameters LLaMA2 model has?")

In [38]:
response.response

'The LLaMA 2 model has four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.'

In [39]:
for src in response.source_nodes:
  print("Node ID\t", src.node_id)
  print("Text\t", src.text)
  print("Score\t", src.score)
  print("-_"*20)

Node ID	 63380d3f-7aff-47cd-b2c1-e4baaed70a7e
Text	 I. Llama 2: Revolutionizing Commercial Use Unlike its predecessor Llama 1, which was limited to research use, Llama 2 represents a major advancement as an open-source commercial model. Businesses can now integrate Llama 2 into products to create AI-powered applications. Availability on Azure and AWS facilitates fine-tuning and adoption. However, restrictions apply to prevent exploitation. Companies with over 700 million active daily users cannot use Llama 2. Additionally, its output cannot be used to improve other language models.  II. Llama 2 Model Flavors Llama 2 is available in four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters. While 7B, 13B, and 70B have already been released, the 34B model is still awaited. The pretrained variant, trained on a whopping 2 trillion tokens, boasts a context window of 4096 tokens, twice the size of its predecessor Llama 1. Meta also released a Llama 2 fine-tuned

In [40]:
query_bundle = hyde("How many parameters LLaMA2 model has?")

In [41]:
hyde_doc = query_bundle.embedding_strs[0]

In [42]:
hyde_doc

'The LLaMA2 model has a total of 12 parameters. These parameters include the weights and biases of the neural network layers, as well as the hyperparameters such as learning rate, batch size, and number of epochs. Additionally, the model may also include regularization parameters such as L1 or L2 regularization coefficients. Overall, these parameters are crucial in determining the performance and behavior of the LLaMA2 model in various machine learning tasks."'